In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import shap
import os
import pickle

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [25]:
df = pd.read_csv('MVP_Candidates.csv')
df.fillna(0)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Pos_No,Age,Tm,G,GS,Seed,...,6.FT+,6.TS+,6.FTr+,6.3PAr+,6.FG Add,6.TS Add,MVP,MVP Rank,MVP Share,Year
0,0,0,Nikola Jokić,C,5,26,DEN,74,74,10,...,105,117,144,55.0,230.9,286.9,1,1,0.875,2021-22
1,1,1,Joel Embiid,C,5,27,PHI,68,68,8,...,105,109,243,47.0,6.5,169.0,0,2,0.706,2021-22
2,2,2,Giannis Antetokounmpo,PF,4,27,MIL,67,67,7,...,93,112,248,49.0,125.1,211.1,0,3,0.595,2021-22
3,3,3,Devin Booker,SG,2,25,PHO,68,68,1,...,112,102,103,84.0,-4.1,32.6,0,4,0.216,2021-22
4,4,4,Luka Dončić,PG,1,22,DAL,65,65,5,...,96,101,140,102.0,-8.9,15.3,0,5,0.146,2021-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,481,481,Bernard King,SF,3,34,WSB,64,64,20,...,103,99,100,30.0,-38.6,-26.0,0,16,0.007,1990-91
482,482,482,Kenny Smith,PG,1,25,HOU,78,78,8,...,110,112,106,164.0,115.5,148.1,0,17,0.005,1990-91
483,483,483,Hakeem Olajuwon,C,5,28,HOU,56,50,8,...,101,103,90,5.0,39.4,31.8,0,18,0.004,1990-91
484,484,484,Tim Hardaway,PG,1,24,GSW,82,82,11,...,105,102,77,198.0,63.4,44.2,0,19,0.001,1990-91


In [26]:
'''
1990-91 to 1998-99
1999-00 to 2011-12
2012-13 to 2021-22
'''
e1 = ('2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15', '2013-14', '2012-13')
e2 = ('2011-12', '2010-09', '2008-09', '2007-08', '2006-07', '2005-06', '2004-05', '2003-04', '2002-03', '2001-02', '2000-01', '1999-00')
e3 = ('1998-99', '1997-98', '1996-97', '1995-96', '1994-95', '1993-94', '1992-93', '1991-92', '1990-91')


df1 = df[(df['Year'] == '2021-22') | (df['Year'] == '2020-21') | (df['Year'] == '2019-20') | (df['Year'] == '2018-19') | (df['Year'] == '2017-18') | (df['Year'] == '2016-17') | (df['Year'] == '2015-16') | (df['Year'] == '2014-15') | (df['Year'] == '2013-14') | (df['Year'] == '2012-13')]


In [27]:
df1

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Pos_No,Age,Tm,G,GS,Seed,...,6.FT+,6.TS+,6.FTr+,6.3PAr+,6.FG Add,6.TS Add,MVP,MVP Rank,MVP Share,Year
0,0,0,Nikola Jokić,C,5,26,DEN,74,74,10,...,105,117,144,55.0,230.9,286.9,1,1,0.875,2021-22
1,1,1,Joel Embiid,C,5,27,PHI,68,68,8,...,105,109,243,47.0,6.5,169.0,0,2,0.706,2021-22
2,2,2,Giannis Antetokounmpo,PF,4,27,MIL,67,67,7,...,93,112,248,49.0,125.1,211.1,0,3,0.595,2021-22
3,3,3,Devin Booker,SG,2,25,PHO,68,68,1,...,112,102,103,84.0,-4.1,32.6,0,4,0.216,2021-22
4,4,4,Luka Dončić,PG,1,22,DAL,65,65,5,...,96,101,140,102.0,-8.9,15.3,0,5,0.146,2021-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,125,125,Kevin Garnett,C,5,36,BOS,68,68,16,...,104,100,87,8.0,2.1,0.6,0,12,0.001,2012-13
126,126,126,Marc Gasol,C,5,28,MEM,80,80,6,...,113,105,134,7.0,-3.8,49.2,0,12,0.001,2012-13
127,127,127,Ty Lawson,PG,1,25,DEN,73,71,4,...,100,103,118,98.0,16.9,31.5,0,12,0.001,2012-13
128,128,128,David Lee,PF,4,29,GSW,79,79,10,...,106,105,102,1.0,52.3,68.1,0,12,0.001,2012-13


In [28]:
df1.drop(df1.columns[[0, 1]], axis = 1, inplace=True)

In [29]:
df1['6.3P+'][108] = 0
''' WHY WAS THIS NaN I HAVE NO CLUE - FILLNA SHOULD HAVE MADE IT ZERO '''
''' ANYWAY PROBLEM SOLVED '''

' ANYWAY PROBLEM SOLVED '

In [30]:
df1.to_csv('ModernEra.csv', index=False)

In [31]:
df1

,Player,Pos,Pos_No,Age,Tm,G,GS,Seed,Win %,1.MP,...,6.FT+,6.TS+,6.FTr+,6.3PAr+,6.FG Add,6.TS Add,MVP,MVP Rank,MVP Share,Year
0,Nikola Jokić,C,5,26,DEN,74,74,10,0.585366,2476,...,105,117,144,55.0,230.9,286.9,1,1,0.875,2021-22
1,Joel Embiid,C,5,27,PHI,68,68,8,0.621951,2297,...,105,109,243,47.0,6.5,169.0,0,2,0.706,2021-22
2,Giannis Antetokounmpo,PF,4,27,MIL,67,67,7,0.621951,2204,...,93,112,248,49.0,125.1,211.1,0,3,0.595,2021-22
3,Devin Booker,SG,2,25,PHO,68,68,1,0.780488,2345,...,112,102,103,84.0,-4.1,32.6,0,4,0.216,2021-22
4,Luka Dončić,PG,1,22,DAL,65,65,5,0.634146,2301,...,96,101,140,102.0,-8.9,15.3,0,5,0.146,2021-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,Kevin Garnett,C,5,36,BOS,68,68,16,0.506173,2022,...,104,100,87,8.0,2.1,0.6,0,12,0.001,2012-13
126,Marc Gasol,C,5,28,MEM,80,80,6,0.682927,2796,...,113,105,134,7.0,-3.8,49.2,0,12,0.001,2012-13
127,Ty Lawson,PG,1,25,DEN,73,71,4,0.695122,2513,...,100,103,118,98.0,16.9,31.5,0,12,0.001,2012-13
128,David Lee,PF,4,29,GSW,79,79,10,0.573171,2907,...,106,105,102,1.0,52.3,68.1,0,12,0.001,2012-13


In [32]:
def calc_metrics(y_test, y_pred):
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    return rmse, r2

In [33]:
# Testing Seasons - 2014-15, 2019-20
# Testing Rows -> 85-96, 27-38

testing_data = df1.iloc[27:39].append(df1.iloc[85:97])

In [34]:
training_data = df1.iloc[0:27].append(df1.iloc[39:85]).append(df1.iloc[97:])

In [35]:
X_train = training_data.drop(['Player', 'Pos', 'Tm', 'MVP', 'MVP Rank', 'MVP Share', 'Year'], axis=1)
Y_train = training_data['MVP Share']

In [36]:
X_test = testing_data.drop(['Player', 'Pos', 'Tm', 'MVP', 'MVP Rank', 'MVP Share', 'Year'], axis=1)
Y_test = testing_data['MVP Share']

In [37]:
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [38]:
final_params = []

In [39]:
param_grid = {'n_estimators': [10,20,30,40,50],
                'learning_rate': [0.01,0.05,0.1,0.2,0.5],
                'max_depth': [3,4,5]}
gb_model = GradientBoostingRegressor()
grid = GridSearchCV(gb_model, param_grid)
grid.fit(scaled_X_train, Y_train)
model = GradientBoostingRegressor(**grid.best_params_)
final_params.append(grid.best_params_)

In [40]:
model.fit(scaled_X_train, Y_train)

GradientBoostingRegressor(n_estimators=30)

In [41]:
Y_pred = model.predict(scaled_X_test)

In [42]:
print(Y_pred)

[0.50002722 0.06538077 0.7362446  0.174077   0.17080655 0.07634642
 0.03364779 0.04890939 0.04406044 0.03364779 0.03364779 0.03364779
 0.44914714 0.34215048 0.03364779 0.76698286 0.50319558 0.31129724
 0.03364779 0.03364779 0.03364779 0.03364779 0.09126603 0.03364779]


In [43]:
initial_results = testing_data[['Player','Year','MVP Share','MVP Rank']]
results = initial_results.copy()
results['Predicted MVP Share'] = pd.Series(Y_pred).values
results = results.sort_values(['Year', 'Predicted MVP Share'], ascending=[False, False]).reset_index(drop=True)
test_season1 = results[(results['Year'] == '2019-20')].reset_index(drop=True)
test_season2 = results[(results['Year'] == '2014-15')].reset_index(drop=True)
test_season1['MVP Rank'] = test_season1.index + 1
test_season2['MVP Rank'] = test_season2.index + 1

In [44]:
print(test_season1)

                   Player     Year  MVP Share  MVP Rank  Predicted MVP Share
0            James Harden  2019-20      0.363         1             0.736245
1   Giannis Antetokounmpo  2019-20      0.952         2             0.500027
2             Luka Dončić  2019-20      0.198         3             0.174077
3           Kawhi Leonard  2019-20      0.166         4             0.170807
4           Anthony Davis  2019-20      0.081         5             0.076346
5            LeBron James  2019-20      0.746         6             0.065381
6          Damian Lillard  2019-20      0.023         7             0.048909
7            Nikola Jokić  2019-20      0.018         8             0.044060
8              Chris Paul  2019-20      0.026         9             0.033648
9           Pascal Siakam  2019-20      0.017        10             0.033648
10           Jimmy Butler  2019-20      0.009        11             0.033648
11           Jayson Tatum  2019-20      0.001        12             0.033648

In [45]:
print(test_season2)

               Player     Year  MVP Share  MVP Rank  Predicted MVP Share
0   Russell Westbrook  2014-15      0.271         1             0.766983
1       Anthony Davis  2014-15      0.156         2             0.503196
2       Stephen Curry  2014-15      0.922         3             0.449147
3        James Harden  2014-15      0.720         4             0.342150
4          Chris Paul  2014-15      0.095         5             0.311297
5       Kawhi Leonard  2014-15      0.001         6             0.091266
6        LeBron James  2014-15      0.425         7             0.033648
7   LaMarcus Aldridge  2014-15      0.005         8             0.033648
8          Marc Gasol  2014-15      0.002         9             0.033648
9       Blake Griffin  2014-15      0.002        10             0.033648
10         Tim Duncan  2014-15      0.001        11             0.033648
11      Klay Thompson  2014-15      0.001        12             0.033648


In [46]:
rmse, r2 = calc_metrics(Y_test, Y_pred)
print("RMSE = ", rmse)
print("R2 = ", r2)

RMSE =  0.2705517883137185
R2 =  0.20098563935789315
